<a href="https://colab.research.google.com/github/Magnet229/Shriram/blob/main/python%20assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
class ValidationException(Exception):
    pass


def validate_file(filename):
    try:
        with open(filename, "r") as file:
            lines = file.readlines()

            # Skip header
            for line_number, line in enumerate(lines[1:], start=2):
                line = line.strip()

                # Skip empty lines
                if not line:
                    continue

                parts = [x.strip() for x in line.split(",")]
                if len(parts) != 2:
                    raise ValidationException(f"Invalid format on line {line_number}: {line}")

                car_id, miles = parts

                # Validate mileage
                if not miles.isdigit():
                    raise ValidationException(
                        f"Invalid mileage '{miles}'"
                    )

            print("All mileage values are valid!")

    except FileNotFoundError:
        raise ValidationException(f"File '{filename}' not found.")


def ex1():
    try:
        validate_file("input.txt")
    except ValidationException as ve:
        print(ve)


# Run example
ex1()


Invalid mileage '32.13'


In [3]:
import csv

def find_total_visits():
    total_visits = 0
    filenames = ["week-1.csv", "week-2.csv", "week-3.csv"]

    for filename in filenames:
        try:
            with open(filename, "r") as file:
                reader = csv.reader(file)
                next(reader)  # Skip header line

                for row in reader:
                    # row = [Name, S, M, T, W, Th, F, S]
                    visits = row[1:]  # skip the name
                    # Convert each day to int and sum
                    total_visits += sum(int(x) for x in visits)

        except FileNotFoundError:
            print(f"Warning: {filename} not found. Skipping this file.")

    return total_visits


def ex2():
    total = find_total_visits()
    print(f"Total visits: {total}.")


# Run the function
ex2()


Total visits: 21.


In [5]:
def count_words(filename):
    try:
        with open(filename, "r") as file:
            text = file.read()

        # Split by whitespace
        words = text.split()

        # Find unique words
        unique_words = set(words)

        # Separate small and large words
        small_words = sorted([w for w in unique_words if len(w) < 3])
        large_words = sorted([w for w in unique_words if len(w) >= 3])

        # Write to files
        with open("small-words.txt", "w") as sfile:
            for word in small_words:
                sfile.write(word + "\n")

        with open("large-words.txt", "w") as lfile:
            for word in large_words:
                lfile.write(word + "\n")

        # Return total number of unique words
        return len(unique_words)

    except FileNotFoundError:
        print(f"File '{filename}' not found.")
        return 0


def ex3():
    total_words = count_words("words.txt")
    print(f"Total words: {total_words}.")


# Run the example
ex3()


Total words: 16.


In [6]:
!pip install boto3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.6 MB/s eta 0:00:00


In [7]:
import boto3

def calculate():
    logs = []
    student_id = "123456"  # 🔹 Replace this with your actual student ID
    filename = f"calculator-log-{student_id}.txt"

    while True:
        first = input("Enter first number: ")

        # Check if user wants to quit
        if first.lower() == 'q':
            break

        second = input("Enter second number: ")

        # Validate numbers
        try:
            num1 = float(first)
            num2 = float(second)
            result = num1 + num2
            print(f"{num1} + {num2} = {result}")
            logs.append(f"{num1} + {num2} = {result}")
        except ValueError:
            print("Invalid input. Please enter valid numbers.")

    # Write all logs to a file
    with open(filename, "w") as file:
        for entry in logs:
            file.write(entry + "\n")

    # Upload file to S3
    try:
        s3 = boto3.client('s3')
        bucket_name = "your-s3-bucket-name"  # 🔹 Replace this with your bucket name
        s3.upload_file(filename, bucket_name, filename)
        print("*** Uploaded to S3 ***")
    except Exception as e:
        print(f"Error uploading to S3: {e}")


def ex4():
    calculate()


# Run example
ex4()


Enter first number: 2
Enter second number: 3
2.0 + 3.0 = 5.0
Enter first number: 5
Enter second number: 6
5.0 + 6.0 = 11.0
Enter first number: q
Error uploading to S3: Unable to locate credentials


In [9]:
from pprint import pprint

def build_car_list():
    car_models = {}  # to map CarId -> Model
    car_list = []    # final list to return

    # Read car-ids.txt file
    with open("car-ids.txt", "r") as f:
        next(f)  # skip header
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = [x.strip() for x in line.split(",")]
            if len(parts) == 2:
                car_id, model = parts
                car_models[car_id] = model

    # Read input.txt for mileage data
    with open("input.txt", "r") as f:
        next(f)  # skip header
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = [x.strip() for x in line.split(",")]
            if len(parts) == 2:
                car_id, miles = parts

                # Check if mileage is an integer (valid)
                if miles.isdigit():
                    car_list.append({
                        "id": int(car_id),
                        "miles": int(miles),
                        "model": car_models.get(car_id, "Unknown")
                    })

    return car_list


def ex5():
    car_list = build_car_list()
    pprint(car_list)


# Run example
ex5()


[{'id': 1, 'miles': 10111, 'model': 'Ford'},
 {'id': 2, 'miles': 30333, 'model': 'Chevy'},
 {'id': 3, 'miles': 40443, 'model': 'Toyota'},
 {'id': 5, 'miles': 60000, 'model': 'Nissan'}]
